# IOU | Intersection Over Union

Question : how do we measure how good a bounding box is?

NOTE: there are two ways to draw bounding boxes depending on what dataset is used, some are drawn from corners and others from mid-points.

<img src="./ref_imgs/IOU_04.png" alt="Description" width="400">

The IOU score will be between 0 and 1.

</br></br></br>

<img src="./ref_imgs/IOU_01.png" alt="Description" width="400">

The green box is the Target/True label for that car object

### Applying IOU

1. Calculate the intersection between the predicted and the correct bboxes
<img src="./ref_imgs/IOU_02.png" alt="Description" width="400">

    1. That is the yellow draw box

2. Then Calculate the Union, which is just the area of both those bounding boxes
<img src="./ref_imgs/IOU_03.png" alt="Description" width="400">
    1. Which is the pink drawn box



### More examples

<img src="./ref_imgs/IOU_05.png" alt="Description" width="250">

image example is 300,550

<img src="./ref_imgs/IOU_06.png" alt="Description" width="250">

Now we look for this yellow box, which is the corner points of the intersection over the two bounding boxes

<img src="./ref_imgs/IOU_07.png" alt="Description" width="400">

<img src="./ref_imgs/IOU_08.png" alt="Description" width="400">

This if for finding the corner points of the yellow intersection box, then we need to calculate the area and the union

In [1]:
import torch

In [ ]:
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    Parameters:
        boxes_pred (python:list): the predicted bounding boxes, shape of (batch_size/N, 4) 4 represents x1, y1, x2, y2
        boxes_labels (python:list): the true labels bounding boxes, shape of (batch_size, 4) 4 represents x1, y1, x2, y2
        
        box_format: (str = "midpoint" or "corners") : how to calculate the bboxes
        
        NOTE: 
            Some bboxes datasets are draw from the midpoint then the height and weight of the bounding box, others are draw from the x1, y1, x2, y2 coordinates
        
    Returns: IOU between two bboxes
    """
    # grab the coordinates
    if box_format== "midpoint":
        # boxes_preds[..., 0:1] is the midpoint
        # boxes_preds[..., 2:3] is the width of that bbox
        # so if we divide it by 2, then well get the top-left x1 coordinate
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2
        
    if box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1] # the ... keeps all the previous dimensions, 
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4] # slice the tensor. but keep its shape, 
        
        box2_x1 = boxes_labels[..., 0:1] # the ... keeps all the previous dimensions, 
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]
    
    
    # compute the yello intersecion box from the images, its the intersection box where the two bboxes share area
    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)
    
    # the .clamp(0) is only for cases when the bounding boxes dont intersect
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    
    
    # calculate the UNION which is the pink drawn box, or just the total area of both the bboxes together
    # the abs() is to make sure we dont have negatives
    box1_area = abs( (box1_x2 - box1_x1) * (box1_y1 - box1_y2))
    box2_area = abs( (box2_x2 - box2_x1) * (box2_y1 - box2_y2))
    
    # Calculate and return the intersection over union, the 1e-6 is for numerical stability
    return intersection / (box1_area + box1_area - intersection + 1e-6)